XLM-RoBERTa 모델을 개체명 인식(NER)을 수행하도록 미세 튜닝하는 방법

참고하는 데이터 포멧 : https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)

In [ ]:
!pip install datasets

In [2]:
from datasets import get_dataset_config_names
xtreme_subsets = get_dataset_config_names('xtreme')
len(xtreme_subsets)  # 서브셋 개수

183

In [6]:
 # Plagiarism Detection Across Languages  :  PAN-X  다양한 언어로 작성된 텍스트에서 표절 감지를 수행하기 위한 데이터
 panx_subsets =  [s for s in xtreme_subsets if s.startswith('PAN')]
 panx_subsets[:5]

['PAN-X.af', 'PAN-X.ar', 'PAN-X.bg', 'PAN-X.bn', 'PAN-X.de']

In [8]:
# 독일어 에 해당하는 코드를 전달
from datasets import load_dataset
load_dataset('xtreme', name = 'PAN-X.de')

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 20000
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs'],
        num_rows: 10000
    })
})

In [11]:
# 스위스 말뭉치 = 스위스에서 사용되는 언어비율로 PAN-X 에서 독일어, 프랑스어 이탈리아어 영어 말뭉치를  샘플링
# 데이터가 불균형하게 만들어진다
# 각 언어를 추적하는  defaultdict 객체를 만든다 , 언어코드를 키로하고  DatasetDict 타입의 PAN-X말뭉치를 값으로 저장
from collections import defaultdict
from datasets import DatasetDict
langs = ['de','fr','it','en']
fracs = [0.629,0.228,0.084,0.059]

In [17]:
# 키가 없는경우 DatasetDict 를 반환
panx_ch = defaultdict(DatasetDict)
for lang,frac in zip(langs,fracs):
  # 다국어 말뭉치를 로드한다
  ds = load_dataset('xtreme', name=f"PAN-X.{lang}")
  # 각 분할을 언어 비율에 따라 다운셈플링하고 섞음
  for split in ds:
    panx_ch[lang][split] = (
        ds[split].shuffle(seed=0).select(range(int(frac*ds[split].num_rows)))
    )

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/20000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [18]:
# 데이터 확인
import pandas as pd
pd.DataFrame({lang: [ panx_ch[lang]['train'].num_rows] for lang in langs },index = ["Number of training examples"] )

,de,fr,it,en
Number of training examples,12580,4560,1680,1180


In [19]:
# 편향된 데이터를 가지고 제로샷 교차 언어 전이를 프랑스어, 이탈리아어, 영어에 수행
# 독일어 말뭉치 셈플을 확인
element =  panx_ch['de']['train'][0]
for key,value in element.items():
  print(f"{key} : {value}")

tokens : ['2.000', 'Einwohnern', 'an', 'der', 'Danziger', 'Bucht', 'in', 'der', 'polnischen', 'Woiwodschaft', 'Pommern', '.']
ner_tags : [0, 0, 0, 0, 5, 6, 0, 0, 5, 5, 6, 0]
langs : ['de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de', 'de']


In [21]:
for key, value in panx_ch['de']['train'].features.items():
  print(f"{key}:{value}")

tokens:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)
ner_tags:Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None), length=-1, id=None)
langs:Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)


In [23]:
tags = panx_ch['de']['train'].features['ner_tags'].feature
tags

ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC'], id=None)

In [25]:
def create_tag_names(batch):
  return{'ner_tags_str' : [ tags.int2str(idx) for idx in batch['ner_tags']] }

panx_de = panx_ch['de'].map(create_tag_names)
panx_de

Map:   0%|          | 0/12580 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

Map:   0%|          | 0/6290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 12580
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 6290
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'langs', 'ner_tags_str'],
        num_rows: 6290
    })
})

In [26]:
de_example =  panx_de['train'][0]
pd.DataFrame([de_example['tokens'], de_example['ner_tags_str']],['Tokens','Tags'] )

,0,1,2,3,4,5,6,7,8,9,10,11
Tokens,2.000,Einwohnern,an,der,Danziger,Bucht,in,der,polnischen,Woiwodschaft,Pommern,.
Tags,O,O,O,O,B-LOC,I-LOC,O,O,B-LOC,B-LOC,I-LOC,O


In [27]:
# 개체명 빈도 계산
from collections import Counter
split2freqs = defaultdict(Counter)
for split,dataset in panx_de.items():
  for row in dataset['ner_tags_str']:
    for tag in row:
      if tag.startswith('B'):
        tag_type = tag.split('-')[1]
        split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient='index')

,LOC,ORG,PER
train,6186,5366,5810
validation,3172,2683,2893
test,3180,2573,3071
